In [1]:
import pandas as pd
import numpy
from __future__ import division

In [2]:
df2_WS2T = pd.read_csv("/Users/asteriag/Desktop/DATASCIENCEPROJECT_WORKFLOW/pythonProject/segment2_WS2_has_duration.csv")

In [3]:
df2_WS2T.head(n=1)

,CASE_ID,WRKFLW_TSK_MAP_GROUPED,WRKFLW_TSK_MAP_DETAILED,PATH_CRDATETIME,ITEM_CRUSER,WRKFLW_ORG_TYPE_CRUSER,STATUS_ORDER,LATEST_STATUS,WRKFLW_PROCESS,QUERY_PART,...,SPK_CAT_TYPE_CD,BROAD_COURSE_LEVEL,DEGREETYPE,LIABILITY_CATEGORY_CD_RECODE,ORIG_LIABILITY_CATEGORY_CD_RECODE,OFFER_RELEASED_FG,OFFER_OUTCOME_CD,OFFER_CONDITIONS,time_used_str,time_used_second
0,914197|528561|809459,Send for Assessment,Send for Assessment,8/11/2019 14:16:43,914197,NaN,11,N,1. App to Assessment,Part 1 - Application ID Workflow,...,MCW,HDCW,Full Degree,AS,AS,Y,$DEC,N,4 days 0 hours 20 minutes 48 seconds,346848.0


In [4]:
numOfWorkflowProcesses = df2_WS2T.groupby('CASE_ID').CASE_ID.count()

count_dic=dict()
for i,v in numOfWorkflowProcesses.items():
    if count_dic.get(v) is None:
        count_dic[v]=[]
        count_dic[v].append(i)
    else:
        count_dic[v].append(i)
#count_dic


In [5]:
len_list=[]
for key in count_dic.keys():
    len_list.append(len(count_dic[key]))

In [6]:
def strfdelta(tdelta, fmt):
    d=dict()
    d["days"],rem1 = divmod(tdelta,60*60*24)
    d["hours"], rem2 = divmod(rem1, 3600)
    d["minutes"], d["seconds"] = divmod(rem2, 60)
    return fmt.format(**d)

In [7]:
list_avgtime_per_key =[]
for key in count_dic.keys():
    id_list = count_dic[key]
    list_sumtime_person = 0
    for id in id_list:
        df = df2_WS2T[df2_WS2T['CASE_ID']== id]
        tsum = df['time_used_second'].sum()
        list_sumtime_person+=tsum
    list_avgtime_per_key.append(int((list_sumtime_person/len(count_dic[key]))))

list_avgtime_per_key_str = []
for i in list_avgtime_per_key:
    list_avgtime_per_key_str.append(strfdelta(i,"{days} days {hours} hours {minutes} minutes {seconds} seconds"))

In [8]:
list_mediantime_per_key = []
for key in count_dic.keys():
    id_list = count_dic[key]
    list_sumtime_per_person = []
    for id in id_list:
        df = df2_WS2T[df2_WS2T['CASE_ID']== id]
        tsum = df['time_used_second'].sum()
        list_sumtime_per_person.append(tsum)
    list_mediantime_per_key.append(numpy.median(list_sumtime_per_person))

list_mediantime_per_key_str = []
for i in list_mediantime_per_key:
    list_mediantime_per_key_str.append(strfdelta(i,"{days} days {hours} hours {minutes} minutes {seconds} seconds"))

In [9]:
list_percent_reach_end = []
for key in count_dic.keys():
    id_list = count_dic[key]
    count_df_end_step = 0
    count_df_id = 0
    for id in id_list:
        df_id = df2_WS2T[df2_WS2T['CASE_ID']== id]
        df_end_step = df_id[df_id['WRKFLW_TSK_MAP_GROUPED'].isin(['Assessment Outcome - Successful','Assessment Outcome - Not Successful']) ]
        count_df_id += df_id['CASE_ID'].count()
        count_df_end_step += df_end_step['CASE_ID'].count()
    list_percent_reach_end.append(str(round((count_df_end_step/count_df_id)*100,2))+ "%")

In [10]:
s2_t2=pd.DataFrame({'Number of Workflow processes in this Workflow Segment':count_dic.keys(),'Number of cases':len_list, 'average time for workflow segment':list_avgtime_per_key_str,'median time for workflow segment':list_mediantime_per_key_str,'percentage of cases reaching to end of workflow segment':list_percent_reach_end})
s2_t2.head(n=10)


,Number of Workflow processes in this Workflow Segment,Number of cases,average time for workflow segment,median time for workflow segment,percentage of cases reaching to end of workflow segment
0,1,115,0 days 0 hours 0 minutes 0 seconds,0.0 days 0.0 hours 0.0 minutes 0.0 seconds,0.0%
1,2,167,19 days 8 hours 11 minutes 2 seconds,10.0 days 23.0 hours 42.0 minutes 6.0 seconds,17.96%
2,3,571,19 days 12 hours 27 minutes 27 seconds,14.0 days 4.0 hours 54.0 minutes 47.0 seconds,30.59%
3,4,334,28 days 0 hours 12 minutes 43 seconds,19.0 days 5.0 hours 22.0 minutes 4.0 seconds,21.33%
4,5,223,33 days 19 hours 54 minutes 53 seconds,21.0 days 0.0 hours 15.0 minutes 38.0 seconds,17.49%
5,6,156,35 days 21 hours 27 minutes 16 seconds,26.0 days 12.0 hours 33.0 minutes 26.0 seconds,14.21%
6,7,78,35 days 6 hours 30 minutes 32 seconds,21.0 days 23.0 hours 0.0 minutes 0.0 seconds,11.54%
7,8,55,48 days 3 hours 50 minutes 12 seconds,25.0 days 23.0 hours 5.0 minutes 5.0 seconds,10.45%
8,9,39,42 days 22 hours 37 minutes 14 seconds,34.0 days 18.0 hours 35.0 minutes 19.0 seconds,9.12%
9,10,23,70 days 2 hours 30 minutes 27 seconds,22.0 days 1.0 hours 59.0 minutes 34.0 seconds,8.26%


In [11]:
s2_t2.to_csv('segment2_WS2_table.csv', index=False)






